Welcome to my NYC Airbnb EDA project - my second EDA project.

We will validate the data to make sure the .csv file is appropriate to work with, and if not then we will clean the data and handle the issues that we might run into with Pandas.

After the validation we analyze the data for any underlying trends and interesting metrics that we want to investigate, and I will present these findings visually through: Matplotlib, Seaborn, and Tableau

In [10]:
import pandas as pd

# Here is where we will investigate and validate the data for completeness and correctness.
# I already did some initial exploration in Data Wrangler to see which columns had null values that needed to be replaced/filled.

filepath = '/Users/andrewhlavacek/Documents/EDA_NYCAirbnb/AB_NYC_2019.csv'
df = pd.read_csv(filepath)

#===============================================================================================================

# Let's see what a row of the .csv looks like
print(df.head(1))

# Check for null values and fill 16 null values
print(df['name'].isnull().sum())
df['name'] = df['name'].fillna('Unknown')

# Check for null values and fill 21 null values
print(df['host_name'].isnull().sum())
df['host_name'] = df['host_name'].fillna('Unknown')

#===============================================================================================================

# Save the updated .csv for later work
df.to_csv('/Users/andrewhlavacek/Documents/EDA_NYCAirbnb/AB_NYC_2019.csv', index=False)




     id                                name  host_id host_name  \
0  2539  Clean & quiet apt home by the park     2787      John   

  neighbourhood_group neighbourhood  latitude  longitude     room_type  price  \
0            Brooklyn    Kensington  40.64749  -73.97237  Private room    149   

   minimum_nights  number_of_reviews last_review  reviews_per_month  \
0               1                  9  2018-10-19               0.21   

   calculated_host_listings_count  availability_365  
0                               6               365  
16
21


Now that we've cleaned the data we can move onto analyzing and visualizing the data so that we can tell a story about Airbnbs in New York

We are going to break down our analysis and visualzation into three categories:

Beginner:

    1.	Which neighborhood group has the most Airbnb listings?
	2.	What is the average price of an Airbnb listing in each borough?
	3.	What is the distribution of room types across NYC?
	4.	Which 5 neighborhoods have the highest average price?
	5.	How many listings are available for at least 300 days a year?

Intermediate:

    1.	Is there a correlation between number of reviews and price?
	2.	What is the average price of listings based on room type and neighborhood group?
	3.	Which hosts have the most listings, and how much do they charge on average?
	4.	How does availability vary by borough across different room types?
	5.	What are the most reviewed listings in Manhattan and their price ranges?

Hard:

	1.	Can we identify clusters of high-priced listings using latitude and longitude (e.g., via KMeans)?
	2.	What is the price elasticity across room types and minimum night stays (does price predict duration)?
	3.	Is there a statistically significant price difference between boroughs (e.g., ANOVA)?
	4.	Can we build a model to predict listing price using features like room type, location, and availability?
	5.	How do availability and number of reviews impact price — controlling for neighborhood and room type (multivariate regression)?

Throughout my analysis I will answer these questions and will be able to compare the results with tableau as well.

In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# === Load Data ===
df = pd.read_csv('/Users/andrewhlavacek/Documents/EDA_NYCAirbnb/AB_NYC_2019.csv')

# === Pre-Calculations ===

# Average price by borough
avg_price_by_neighborhood = df.groupby('neighbourhood_group')['price'].mean().sort_values(ascending=False)
# print(avg_price_by_neighborhood)

# Top 5 neighborhoods by average price
top_5_neighborhoods = df.groupby('neighbourhood')['price'].mean().sort_values(ascending=False).head(5)
# print(top_5_neighborhoods)

# Listings available at least 300 days per year
day_listing_300 = df['availability_365'] >= 300
# print(day_listing_300)

# === Create Plot ===

# Set the style
plt.style.use('seaborn-v0_8')
fig, axes = plt.subplots(3, 2, figsize=(20, 25))
fig.suptitle('NYC Airbnb Analysis - Key Findings (Beginner Level)', fontsize=20, y=0.95)

# === 1. Number of listings by borough === 
neighborhood_counts = df['neighbourhood_group'].value_counts().reset_index()
neighborhood_counts.columns = ['neighbourhood_group', 'count']
sns.barplot(
    data=neighborhood_counts,
    x='neighbourhood_group',
    y='count',
    ax=axes[0, 0],
    palette='viridis'
)
axes[0, 0].set_title('Number of Listings by Borough')
axes[0, 0].set_xlabel('Borough')
axes[0, 0].set_ylabel('Number of Listings')
axes[0, 0].tick_params(axis='x', rotation=45)

# === 2. Average price by borough ===
avg_price_df = avg_price_by_neighborhood.reset_index()
avg_price_df.columns = ['neighbourhood_group', 'average_price']
sns.barplot(
    data=avg_price_df,
    x='neighbourhood_group',
    y='average_price',
    ax=axes[0, 1],
    palette='magma'
)
axes[0, 1].set_title('Average Price by Borough')
axes[0, 1].set_xlabel('Borough')
axes[0, 1].set_ylabel('Average Price ($)')
axes[0, 1].tick_params(axis='x', rotation=45)

# === 3. Room type distribution ===
room_type_counts = df['room_type'].value_counts()
colors = sns.color_palette('husl', n_colors=len(room_type_counts))
axes[1, 0].pie(room_type_counts, labels=room_type_counts.index, autopct='%1.1f%%', colors=colors)
axes[1, 0].set_title('Distribution of Room Types')

# === 4. Top 5 neighborhoods by average price ===
top_5_df = top_5_neighborhoods.reset_index()
top_5_df.columns = ['neighbourhood', 'average_price']
sns.barplot(
    data=top_5_df,
    x='neighbourhood',
    y='average_price',
    ax=axes[1, 1],
    palette='rocket'
)
axes[1, 1].set_title('Top 5 Neighborhoods by Average Price')
axes[1, 1].set_xlabel('Neighborhood')
axes[1, 1].set_ylabel('Average Price ($)')
axes[1, 1].tick_params(axis='x', rotation=45)

# === 5. Availability distribution (highlighting 300+ days) ===
sns.histplot(
    data=df,
    x='availability_365',
    bins=50,
    ax=axes[2, 0],
    color='darkblue',
    alpha=0.6
)
axes[2, 0].axvline(x=300, color='red', linestyle='--', label='300 days threshold')
axes[2, 0].set_title('Distribution of Listing Availability')
axes[2, 0].set_xlabel('Days Available per Year')
axes[2, 0].set_ylabel('Number of Listings')
axes[2, 0].legend()

# === Remove unused subplot ===
axes[2, 1].remove()

# === Final Touches ===
fig.text(0.5, 0.02, 'Data Source: Inside Airbnb (via Kaggle)', ha='center', fontsize=12)
plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.savefig('nyc_airbnb_beginner_analysis.png', dpi=300)
plt.show()

/var/folders/xz/p2ff3pyj7ld80r8bfv3x3lk40000gn/T/ipykernel_21164/3714363476.py:32: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
/var/folders/xz/p2ff3pyj7ld80r8bfv3x3lk40000gn/T/ipykernel_21164/3714363476.py:47: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
/var/folders/xz/p2ff3pyj7ld80r8bfv3x3lk40000gn/T/ipykernel_21164/3714363476.py:68: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

filepath = '/Users/andrewhlavacek/Documents/EDA_NYCAirbnb/AB_NYC_2019.csv'
df = pd.read_csv(filepath)

# Create a figure with subplots
fig, axes = plt.subplots(3, 2, figsize=(20, 25))
plt.suptitle('NYC Airbnb Intermediate Analysis', fontsize=16, y=0.98)

# 1. Correlation between reviews and price
correlation = df['number_of_reviews'].corr(df['price'])
axes[0,0].scatter(df['number_of_reviews'], df['price'], alpha=0.5, color='darkblue')
axes[0,0].set_xlabel('Number of Reviews')
axes[0,0].set_ylabel('Price')
axes[0,0].set_title(f'Reviews vs Price Correlation (r={correlation:.2f})')

# 2. Average price by room type and neighborhood
avg_price_by_room_type = df.groupby(['room_type', 'neighbourhood_group'])['price'].mean().reset_index()
sns.barplot(
    x='room_type', 
    y='price', 
    hue='neighbourhood_group', 
    data=avg_price_by_room_type,
    ax=axes[0,1],
    palette='husl'
)
axes[0,1].set_title('Average Price by Room Type and Borough')
axes[0,1].set_xlabel('Room Type')
axes[0,1].set_ylabel('Average Price ($)')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Top 10 hosts average prices
host_listing_count = df['host_id'].value_counts().reset_index()
host_listing_count.columns = ['host_id', 'listing_count']
avg_price_by_host = df.groupby('host_id')['price'].mean().reset_index()
top_10_hosts = host_listing_count.sort_values(by='listing_count', ascending=False).head(10)
top_10_hosts = top_10_hosts.merge(avg_price_by_host, on='host_id')

sns.barplot(
    x='host_id', 
    y='price', 
    data=top_10_hosts,
    ax=axes[1,0],
    palette='rocket'
)
axes[1,0].set_title('Average Price for Top 10 Hosts')
axes[1,0].set_xlabel('Host ID')
axes[1,0].set_ylabel('Average Price ($)')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Availability by room type and borough
availability_by_room_type = df.groupby(['room_type', 'neighbourhood_group'])['availability_365'].mean().reset_index()
sns.barplot(
    x='room_type', 
    y='availability_365', 
    hue='neighbourhood_group', 
    data=availability_by_room_type,
    ax=axes[1,1],
    palette='mako'
)
axes[1,1].set_title('Average Availability by Room Type and Borough')
axes[1,1].set_xlabel('Room Type')
axes[1,1].set_ylabel('Average Availability (Days)')
axes[1,1].tick_params(axis='x', rotation=45)

# 5. Most reviewed listings in Manhattan
manhattan_listings = df[df['neighbourhood_group'] == 'Manhattan']
top_8_most_reviewed = manhattan_listings.sort_values(by='number_of_reviews', ascending=False).head(8)
sns.barplot(
    x='name', 
    y='price', 
    data=top_8_most_reviewed,
    ax=axes[2,0],
    palette='viridis'
)
axes[2,0].set_title('Top 8 Most Reviewed Manhattan Listings')
axes[2,0].set_xlabel('Listing Name')
axes[2,0].set_ylabel('Price ($)')
axes[2,0].tick_params(axis='x', rotation=90)

# Remove unused subplot
axes[2,1].remove()

# Add overall title and adjust layout
plt.tight_layout()
plt.show()

# save the figure as a png 
plt.savefig('nyc_airbnb_intermediate_analysis.png')


/var/folders/xz/p2ff3pyj7ld80r8bfv3x3lk40000gn/T/ipykernel_21164/3899623524.py:41: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
/var/folders/xz/p2ff3pyj7ld80r8bfv3x3lk40000gn/T/ipykernel_21164/3899623524.py:71: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


<Figure size 800x550 with 0 Axes>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

filepath = '/Users/andrewhlavacek/Documents/EDA_NYCAirbnb/AB_NYC_2019.csv'
df = pd.read_csv(filepath)

# Now it's time to answer the HARD questions

#	1.	Can we identify clusters of high-priced listings using latitude and longitude (e.g., via KMeans)?
#	2.	What is the price elasticity across room types and minimum night stays (does price predict duration)?


#	3.	Is there a statistically significant price difference between boroughs (e.g., ANOVA)?



#	4.	Can we build a model to predict listing price using features like room type, location, and availability?
#	5.	How do availability and number of reviews impact price — controlling for neighborhood and room type (multivariate regression)?

